In [2]:
import numpy  as no
import os
import json

In [6]:
group_path = 'group4'
gt_path = os.path.join('./data_origin/newgt', group_path)

In [7]:
annos={}
annos['imgs']={}

for file in os.listdir(gt_path):
    imgid = file.split('.')[0]
    file_name = imgid + '.jpg'
    path = group_path + '/' + file_name
    annos['imgs'][imgid] = {}
    annos['imgs'][imgid]['id'] = imgid
    annos['imgs'][imgid]['path'] = path
    annos['imgs'][imgid]['objects'] = []
    with open(os.path.join(gt_path, file), 'r') as f:
        lines = f.readlines()
    for line in lines:
        line = line.strip().split(' ')
        category = line[1][0]

        bbox={}
        bbox['xmin'] = float(line[2])
        bbox['ymin'] = float(line[3])
        bbox['xmax'] = float(line[4])
        bbox['ymax'] = float(line[5])

        annos['imgs'][imgid]['objects'].append({'bbox':bbox, 'category':category})
#         print(line)

In [8]:
output_path = './data_cleared/annos/annos_' + group_path + '.json'
with open(output_path, 'w') as f:
    json.dump(annos, f)
    f.close()

# Resize
将annotation的检测框resize到512*512

In [ ]:
annos={}
annos['imgs']={}

for file in os.listdir(gt_path):
    imgid = file.split('.')[0]
    annos['imgs'][imgid] = {}
    annos['imgs'][imgid]['id'] = imgid
    annos['imgs'][imgid]['path'] = os.path.join(group_path, file)
    annos['imgs'][imgid]['objects'] = []
    with open(os.path.join(gt_path, file), 'r') as f:
        lines = f.readlines()
    for line in lines:
        line = line.strip().split(' ')
        category = line[1][0]

        bbox={}
        bbox['xmin'] = float(line[2]) / (1280/512)
        bbox['xmax'] = float(line[4]) / (1280/512)
        bbox['ymin'] = float(line[3]) / (1024/512)
        bbox['ymax'] = float(line[5]) / (1024/512)

        annos['imgs'][imgid]['objects'].append({'bbox':bbox, 'category':category})
#         print(line)

In [ ]:
output_path = 'annos_' + group_path + '.json'
with open(output_path, 'w') as f:
    json.dump(annos, f)
    f.close()

# Generate annotations txt
生成rezise后对应的groud_truth的TXT文件

In [17]:
input_dir = './data_origin/newgt'
output_dir = 'data_resize/newgt'
group_path = 'group1'
gt_path = os.path.join(input_dir, group_path)
output_path = os.path.join(output_dir, group_path)

if not os.path.exists(output_path):
    os.mkdir(output_path)

for file in os.listdir(gt_path):
    with open(os.path.join(gt_path, file), 'r') as fin:
        lines = fin.readlines()
    with open(os.path.join(output_path, file), 'w') as fout:
        for line in lines:
            line = line.strip().split(' ')
            category = line[1][0]
            x_min = str(float(line[2]) / (1280/512))
            x_max = str(float(line[4]) / (1280/512))
            y_min = str(float(line[3]) / (1024/512))
            y_max = str(float(line[5]) / (1024/512))
            
            res = file+' '+category+" "+x_min+' '+y_min+' '+x_max+' '+y_max+'\n'
            fout.write(res)

# Counter
统计annotations下每个类别的数目

In [13]:
import os
import json
from collections import Counter

In [27]:
datadir='../data'
filedir=os.path.join(datadir, 'annotations.json')
annos=json.loads(open(filedir).read())

In [28]:
class_counter=Counter()
classes=set()
for imgid in annos['imgs']:
    for boxes in annos['imgs'][imgid]['objects']:
        class_counter[boxes['category']]+=1
        classes.add(boxes['category'])
classes=list(classes)

In [29]:
print(class_counter)
print(classes)

Counter({'z': 9990, 'd': 5798, 's': 4313, 'j': 3097, 'l': 683})
['z', 's', 'j', 'd', 'l']


# Merge
将两个annotation合并起来

In [17]:
import json
import os

In [18]:
anno1_path = '../data/annotation_aug.json'
anno2_path = '../data/annotation_sample.json'

In [23]:
a1 = json.load(open(anno1_path, 'r'))
a2 = json.load(open(anno2_path, 'r'))

print(len(a1['imgs']))
print(len(a2['imgs']))

7832
3000


In [24]:
a2['imgs'].update(a1['imgs'])

print(len(a2['imgs']))

10832


In [26]:
json.dump(a2, open('../data/annotations.json', 'w'))

# Transform
将Tsinghua_tencent_100K的标注转换成三类

In [ ]:
import json
import os

In [ ]:
datadir='../data'
sign_map = {'i':'s', 'p':'z', 'w':'j'}
anno_path = 'annotation.json'
target_path = 'annotation_3class.json'

In [ ]:
anno_file=os.path.join(datadir, anno_path)
annos=json.loads(open(anno_file).read())

In [ ]:
for id in annos['imgs']:
    for bi, box in enumerate(annos['imgs'][id]['objects']):
        category = box['category']
        new_category = sign_map[category[0]]
        annos['imgs'][id]['objects'][bi]['category'] = new_category

In [ ]:
with open(os.path.join(datadir, target_path), 'w') as f:
    json.dump(annos, f)
    f.close()